# Libraries import
- Data wrangling and indicators generate
- api to trade (Binance)
- api llm (openai)

In [4]:
import os
import pandas as pd
import numpy as np
import schedule
import ta
import time
import re
from datetime import datetime
from dotenv import load_dotenv
import json
from binance.client import Client
import openai
from openai import OpenAI

load_dotenv();

# APIS Configuration

In [11]:
mclient = Client(os.getenv("BINANCE_API_KEY"), api_secret = os.getenv("BINANCE_API_SECRET"),testnet=True)

In [12]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# We build functions
- Data download
- Calculation of indicators
- Signals
- Operations executor
- Log function
- Agent executor

In [13]:
def get_data(ticker,period):  
    klines = mclient.get_historical_klines(ticker, period, limit = 1000) #since, to
    df = pd.DataFrame(klines)
    df = df.iloc[:,[0,1,2,3,4,5,7,8,9,10]] 
    df.columns = ['Time','Open','High','Low','Close','VolumeCurr','VolumeUSD','Trades','VolumeBase','Volume2']
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df=df.astype("float")
    df['ASSET'] = f"{ticker}"[0:3] 
    return df

In [14]:
def get_indicators(df):
    df['var_pct'] = df['Close'].pct_change()
    df = df.dropna(subset=['var_pct']).copy()
    df['SMA_20'] = ta.trend.sma_indicator(df['Close'], window=20)
    df['SMA_50'] = ta.trend.sma_indicator(df['Close'], window=50)
    df['RSI'] = ta.momentum.rsi(df['Close'], window=14)
    df['MACD'] = ta.trend.macd(df['Close'])
    df['MACD_Signal'] = ta.trend.macd_signal(df['Close'])

    df['MA20'] = df['Close'].rolling(20).mean()
    df['MA100'] = df['Close'].rolling(100).mean()
    
    df['Signal'] = 0.0
    df.loc[df['MA20'] > df['MA100'], 'Signal'] = 1.0
    df.loc[df['MA20'] < df['MA100'], 'Signal'] = -1.0

    df['Plog_ret'] = np.log(df.Close) - np.log(df.Close.shift(1))
    df['pup_6std'] = df.Close*(1+(df.Plog_ret.mean() + 6*df.Plog_ret.std()))
    df['pdown_6std'] = df.Close*(1+(df.Plog_ret.mean() - 6*df.Plog_ret.std()))

    df['std_dev'] = df['Close'].rolling(window=20).std()
    df['upper_band'] = df['SMA_20'] + (6 * df['std_dev'])
    df['lower_band'] = df['SMA_20'] - (6 * df['std_dev'])
    return df

In [15]:
def get_signal(df):

    buy_signals = (df['Signal'].shift(1) == -1.0) & (df['Signal'] == 1.0)
    sell_signals = (df['Signal'].shift(1) == 1.0) & (df['Signal'] == -1.0)

    df.loc[buy_signals, 'Trade_Signal'] = 'BUY'

    df.loc[sell_signals, 'Trade_Signal'] = 'SELL'
    return df

In [16]:
mbalance = mclient.get_asset_balance(asset='USDT')

In [35]:
def get_chat(system_prompt, user_message):
    completion = openai_client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": 'system', 'content': system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    response = completion.choices[0].message.content.strip()
    try:
        #print(f"repr(response): {repr(response)}")
        response_json = json.loads(response)
        #print(repr(response_json))
        print(f"Reasoning: {response_json.get('summary')}")  
        #response_json.get("summary")
        return response_json.get("direction").lower(), response_json.get("size")
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e.doc)
        print(f"Message: {e.msg}")
        print(f"Line: {e.lineno}")
        print(f"Column: {e.colno}")
        print(f"Position: {e.pos}")        
        return None, None

In [21]:
def get_trade(direction, asset):
    symbol_to_find = f"{asset}USDT"
    
    # Get symbol information
    symbol_info = mclient.get_symbol_info(symbol_to_find)
    
    # 1. Get LOT_SIZE filters
    lot_size_filter = next(filter(lambda x: x['filterType'] == 'LOT_SIZE', symbol_info['filters']))
    min_qty = float(lot_size_filter['minQty'])
    step_size = float(lot_size_filter['stepSize'])
    
    # 2. Calculate base amount based on current price
    current_price = float(next(t['price'] for t in mclient.get_all_tickers() if t['symbol'] == symbol_to_find))
    raw_amount = 100 / current_price
    
    # 3. Calculate required decimals based on step_size
    def get_precise_decimals(step_size):
        step_str = str(step_size)
        if 'e' in step_str.lower():
            # Handle scientific notation
            base, exponent = step_str.lower().split('e')
            return abs(int(exponent))
        else:
            # Handle normal decimals
            return len(step_str.split('.')[1]) if '.' in step_str else 0
    
    decimal_places = get_precise_decimals(step_size)
    
    # 4. Calculate number of steps and adjust amount
    steps = int(raw_amount / step_size)  # Truncate to complete steps
    adjusted_amount = steps * step_size
    
    # 5. Ensure we meet minimum quantity
    adjusted_amount = max(adjusted_amount, min_qty)
    
    # 6. Format amount with correct decimals
    adjusted_amount_str = f"{adjusted_amount:.{decimal_places}f}"
    
    # 7. Convert back to float for operations
    final_amount = float(adjusted_amount_str)
    
    # 8. Final validation
    if final_amount < min_qty:
        raise ValueError(f"⚠️ Amount {final_amount} below minimum {min_qty}")
    
    try:
        if direction == 'buy':
            order = mclient.order_market_buy(
                symbol=symbol_to_find,
                quantity=adjusted_amount_str,  # Use formatted string
                type='MARKET'
            )
            print("Purchase order executed:", order)
            return order
            
        elif direction == 'sell':
            order = mclient.order_market_sell(
                symbol=symbol_to_find,
                quantity=adjusted_amount_str,  # Use formatted string
                type='MARKET'
            )
            print("Sell order executed:", order)
            
        else:
            print('No order is placed')
            
    except Exception as e:
        print(f"Error executing order: {str(e)}")
        raise

In [25]:
def run_agent(ticker,period):
    
    # Get initial data
    try:
        # Check if ticker exists in available pairs
        available_tickers = [t['symbol'] for t in mclient.get_all_tickers()]
        
        if ticker not in available_tickers:
            print(f"❌ Trading pair {ticker} not available")
            return None
    except Exception as e:
        print(f"❌ Error validating {ticker}: {str(e)}")
        return None

    data = get_data(ticker, period)
    trading_asset = ticker.split('USDT')[0]
    data = get_indicators(data)
    data = get_signal(data)

    system_prompt = (
        '#Instructions: \n'
        'Act as an experienced cryptocurrency trader who helps users identify market trends.\n'
        'You help people who want to trade according to their strategy, however, they want an experts opinion regarding their decision to buy or sell, which is not based only on their strategy but also on your expertise.\n'
        f'You are provided with a {mbalance} that represents the total amount available that the user has to trade, on the asset that the user provides you.\n'
        'Make the decision to buy or sell the asset according to the users strategy and also on the pool of technical indicators that will be shared with you.\n'
        'The value in USDT of each individual operation must not exceed the balance and the operations must have a size that allows trading with what is available in the balance and also allows handling market volatility.\n'
        f'The value in USDT of each individual operation must not exceed 100 and the operations must have a size that allows trading with what is available in the {mbalance} balance and also allows handling market volatility.\n'
        'Do not suggest or provide a reasoning for the order when the suggested order size (both for new positions and for adding to existing positions) is less than 100 USDT.\n'
        '# Available options:\n'
        '- close a position only when there is a moving average crossover opposite to the crossover with which a trade is opened\n'
        '- there must not be 2 open positions in the same week\n'
        '# Fields for each option:\n'
        f'- asset: the asset to be trade is {trading_asset}\n'
        '- direction: the direction to trade\n'
        '\t- example: buy, sell\n'
        '- size: the size of the trade denominated in USDT. It should be greater than 10 and should not use all of the balance leaving sufficient funds available to risk management.\n'
        '- reasoning: the reasoning for the decision\n'
        'if in the variable there is a BUY signal then you should take a long or buy position if the other indicators you check confirm it\n'
        'if in the variable there is a SELL signal then you should take a short or sell position if the other indicators you check confirm it\n'
        'Please provide ONLY the response in the following JSON format without any additional text or markdown:\n'
        '{\n'
        f'  "asset": "{trading_asset}",\n'
        '  "direction": "buy",\n'
        '  "size": "100",\n'
        f'  "summary": "Based on the provided indicators, it is recommended to buy {trading_asset}.\n'
        '}'
    )

    user_message = (
        '# Instructions:\n'
        'Here are some details about me, can you help me make decisions about what position I should take?\n'
        'I need to place a trade weekly\n'
        '# Available Balance\n'
        'The available balance to trade is however each trade must be 100 USDT\n'
        'This balance can be used to place new orders or modify existing positions.\n'
        'Always leave a fraction of the balance total as a safety cushion for unforeseen volatility.\n'
        'The balance is shared by all positions, so its important to keep track of the available value and adjust your position sizes accordingly.\n'
        '# This is the most recent information I want to base my decisions on:\n'
        'My strategy is based on the crossover of moving averages, whose signal of trade that is this can be BUY or SELL, but as I understand that this is a decision that can be influenced by other indicators, I ask you based on your experience to evaluate the following indicators\n'
        f"""
        Technical analysis data for the asset {trading_asset}:
        - SMA 20: {data['SMA_20'].iloc[-1]}
        - SMA 50: {data['SMA_50'].iloc[-1]}
        - RSI: {data['RSI'].iloc[-1]}
        - MACD: {data['MACD_Signal'].iloc[-1]}
        - Short mean reversion: {data['upper_band'].iloc[-1]}
        - Long mean reversion: {data['lower_band'].iloc[-1]}
        """
        'In addition to all the advice you give me, I need you to always summarize your conclusion with these points **Asset**, **Direction** (here the options are buy, sell, hold) and **Operation size**, in case it is to hold and a new position is not generated, then suggest 0 for size'    
    )

    direction, size = get_chat(system_prompt, user_message)
    if direction:
        if direction in ["buy", "sell"]:
            print(f"direction: {direction}, asset: {trading_asset}, size: {size}")
            get_trade(direction, trading_asset)
        elif direction == 'hold':
            print("HODL!")
        else:
            print("No valid signal (buy, sell, or hold) detected.:", direction)
    else:
        print(f"Could not retrieve a valid response from the model. {direction}")

In [36]:
run_agent('BTCUSDT','1h')

Reasoning: Based on the provided indicators, the moving average (SMA 20 crossing above SMA 50) suggests a bullish trend. The RSI indicates moderate strength, and MACD is positive, reinforcing the buy signal.
direction: buy, asset: BTC, size: 100
Purchase order executed: {'symbol': 'BTCUSDT', 'orderId': 889673, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJbb7abd8eeb858c541588e2', 'transactTime': 1736574701743, 'price': '0.00000000', 'origQty': '0.00106000', 'executedQty': '0.00106000', 'origQuoteOrderQty': '0.00000000', 'cummulativeQuoteQty': '99.97015820', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1736574701743, 'fills': [{'price': '94311.47000000', 'qty': '0.00106000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'tradeId': 307536}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}


# Scheduled Agent Execution
This code runs a trading agent for BTCUSDT pair every minute, executing exactly 3 times before stopping. 
It uses a simple counter approach with the schedule library for precise timing control.

In [45]:
"""
for i in range(3):  # Will run 3 times
    run_agent('BTCUSDT', '1h')
    print(f"Execution {i+1} of 3 completed")
    if i < 2:  # Don't sleep after the last execution
        time.sleep(60)  # Wait 1 minute between executions
        
print("All 3 executions completed!")
"""

'\nfor i in range(3):  # Will run 3 times\n    run_agent(\'BTCUSDT\', \'1h\')\n    print(f"Execution {i+1} of 3 completed")\n    if i < 2:  # Don\'t sleep after the last execution\n        time.sleep(60)  # Wait 1 minute between executions\n        \nprint("All 3 executions completed!")\n'